# Stacked AutoEncoders

## Downloading the dataset

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip

## Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [3]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [4]:
training_set = pd.read_csv('ml-1m/training_set.csv')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-1m/test_set.csv')
test_set = np.array(test_set, dtype='int')

## Getting the number of users and movies


In [5]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [6]:
def convert(data):
	new_data = []
	for id_users in range(1, nb_users + 1):
		id_movies = data[:, 1] [data[:, 0] == id_users]
		id_ratings = data[:, 2] [data[:, 0] == id_users]
		ratings = np.zeros(nb_movies)
		ratings[id_movies - 1] = id_ratings
		new_data.append(list(ratings))
	return new_data


training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [8]:
class SAE(nn.Module): # Inheritance of class nn.Module
	def __init__(self, ):
		super(SAE, self).__init__()			# Gets all the inherited classes and methods 
		self.fc1 = nn.Linear(nb_movies, 20) # First hidden leayer with 20 neurons
		self.fc2 = nn.Linear(20, 10)		# Second hidden layer with 10 neurons
		self.fc3 = nn.Linear(10, 20)		# Thir hidden layer with 20 neurons
		self.fc4 = nn.Linear(20, nb_movies) # Output layer
		self.activation = nn.Sigmoid()
	def forward(self, x):
		x = self.activation(self.fc1(x)) # Encoding
		x = self.activation(self.fc2(x)) # Encoding
		x = self.activation(self.fc3(x)) # Decoding
		x = self.fc4(x)					 # Output vector
		return x


sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

## Training the SAE


In [13]:
nb_epochs = 50
for epoch in range(1, nb_epochs + 1):
	train_loss = 0
	s = 0.
	for id_user in range(nb_users):
		input = Variable(training_set[id_user]).unsqueeze(0)	# Adds a dimension to input vector
		target = input.clone()
		if torch.sum(target.data > 0) > 0:	# Check if user has rated at least one movie
			output = sae(input)
			target.require_grad = False # Memory ptimization
			output[target == 0] = 0		# Memory optimization
			loss = criterion(output, target)
			mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
			loss.backward()		# Decided to increse or decrese weights
			train_loss += np.sqrt(loss.data * mean_corrector)
			s += 1.
			optimizer.step()	# Decides how intense is the adjustment for weights
	print(f"Epoch: {epoch} Loss: {train_loss/s}")

Epoch: 1 Loss: 1.3470802307128906
Epoch: 2 Loss: 1.0101701021194458
Epoch: 3 Loss: 0.9898726940155029
Epoch: 4 Loss: 0.9832795858383179
Epoch: 5 Loss: 0.9802073836326599
Epoch: 6 Loss: 0.978493869304657
Epoch: 7 Loss: 0.9773132801055908
Epoch: 8 Loss: 0.9765602946281433
Epoch: 9 Loss: 0.9758267402648926
Epoch: 10 Loss: 0.9755949974060059
Epoch: 11 Loss: 0.9751777052879333
Epoch: 12 Loss: 0.9749517440795898
Epoch: 13 Loss: 0.9748559594154358
Epoch: 14 Loss: 0.974542498588562
Epoch: 15 Loss: 0.9744098782539368
Epoch: 16 Loss: 0.9743704795837402
Epoch: 17 Loss: 0.974302351474762
Epoch: 18 Loss: 0.9740024209022522
Epoch: 19 Loss: 0.973993182182312
Epoch: 20 Loss: 0.9739540219306946
Epoch: 21 Loss: 0.9737952351570129
Epoch: 22 Loss: 0.9736430644989014
Epoch: 23 Loss: 0.9738006591796875
Epoch: 24 Loss: 0.9732851982116699
Epoch: 25 Loss: 0.9725657105445862
Epoch: 26 Loss: 0.9721838235855103
Epoch: 27 Loss: 0.9715476036071777
Epoch: 28 Loss: 0.9706843495368958
Epoch: 29 Loss: 0.969968378543853

## Loss represents the average difference of our predicted rating and the actual rating. Movie ratings are from 1 to 5

## Testing the SAE


In [49]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
	input = Variable(training_set[id_user]).unsqueeze(0)
	target = Variable(test_set[id_user]).unsqueeze(0)
	if torch.sum(target.data > 0) > 0:
		output = sae(input)
		target.require_grad = False
		output[target == 0] = 0
		loss = criterion(output, target)
		mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
		test_loss += np.sqrt(loss.data * mean_corrector)
		s += 1.
print(f"Test Loss: {test_loss/s}")

Test Loss: 0.9460129737854004


# On average our module was wrong by < 1 star rating